In [40]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LeakyReLU, Flatten, MaxPooling1D, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense, AveragePooling1D, GlobalAveragePooling1D

In [2]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
%matplotlib tk

In [3]:
import os, os.path, sys

In [4]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\X"

In [5]:
dir = os.listdir(path)

Ucitavanje fajlova

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.8

In [8]:
percent_val = 0.2

Konkatenacija podataka koji se posle koriste za adekvatnu raspodelu na skup za treniranje i validaciju

In [9]:
train_files_names = file_names[:int(percent_train*len(file_names))]
#test_file_names = file_names[int(percent_train*len(file_names))::]

Ucitavanje zadate trajektorije

In [10]:
x_command = np.asarray(pd.read_csv(r"C:\Users\mata2\Desktop\master\podaci\xCmd.txt",header=None).astype('int'))

In [11]:
train_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)
# test_ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
#                               for file in test_file_names], axis=0)

Ucitavanje stadardnih devijacija i strednih vrednosti

In [12]:
std_dev = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k\std_dev.txt", 'r') as lines:
    for line in lines:
        std_dev.append(float(line.strip('\n')))

In [13]:
sr_vrednost = []
with open(r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\odstupanja\0k\sr_vrednost.txt", 'r') as lines:
    for line in lines:
        sr_vrednost.append(float(line.strip('\n')))

Raspodela podataka kako bi se izbegla fiktivna stanja

In [15]:
for j in range(len(train_files_names)):
    exec(f"razlika_trening_{j} = []")
    for i in range(len(x_command)):
        exec(f"razlika_trening_{j}.append(x_command[i] - train_[j*len(x_command)+i])")
    exec(f'razlika_trening_{j} = np.asarray(razlika_trening_{j})')        

In [14]:
def create_timeseries_split(data, time_steps):

    # Get the number of samples in the data
    num_samples = len(data) - time_steps

    # Create empty arrays to store features and target values
    x_train = np.zeros((num_samples, time_steps, 1))
    y_train = np.zeros((num_samples, 1))

    # Loop through the data and create features and target values
    for i in range(num_samples):
        # Extract a slice of data for the current feature
        x_train[i] = data[i:i+time_steps, :]

        # The target value is the next value after the feature
        y_train[i] = data[i+time_steps, 0]

    return x_train, y_train

In [16]:
time_steps = 8
input_shape = (time_steps,1)

In [17]:
for i in range(len(train_files_names)):
    exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, time_steps)')
    
train_X = np.asarray(train_X_0)
for i in range(1,len(train_files_names)):
    exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

train_y = np.asarray(train_y_0)
for i in range(1,len(train_files_names)):
    exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)') 

In [19]:
val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

In [20]:
train_X = train_X[:len(train_X)-val_x.shape[0]]
train_y = train_y[:len(train_y)-val_x.shape[0]]

# TESTIRANJE ARHITEKTURE i LR/BATCH

In [72]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

modelMSE = Sequential()
modelMSE.add(Input(shape=(time_steps, 1)))
modelMSE.add(Conv1D(filters=32, kernel_size=2,padding='same', activation=LeakyReLU()))
#modelMSE.add(Conv1D(filters=32, kernel_size=2, activation=LeakyReLU()))
modelMSE.add(MaxPooling1D(pool_size=3))
modelMSE.add(Conv1D(filters=64, kernel_size=2,padding='same', activation=LeakyReLU()))
#modelMSE.add(Conv1D(filters=64, kernel_size=2, activation=LeakyReLU()))
modelMSE.add(GlobalAveragePooling1D())
modelMSE.add(Flatten())
modelMSE.add(Dense(128, activation=LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [73]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 256, shuffle=False, verbose = 2)

Epoch 1/10
4575/4575 - 17s - loss: 12.0050 - 17s/epoch - 4ms/step
Epoch 2/10
4575/4575 - 16s - loss: 8.2173 - 16s/epoch - 3ms/step
Epoch 3/10
4575/4575 - 16s - loss: 7.5679 - 16s/epoch - 4ms/step
Epoch 4/10
4575/4575 - 17s - loss: 7.1447 - 17s/epoch - 4ms/step
Epoch 5/10
4575/4575 - 17s - loss: 6.8619 - 17s/epoch - 4ms/step
Epoch 6/10
4575/4575 - 16s - loss: 6.6760 - 16s/epoch - 4ms/step
Epoch 7/10
4575/4575 - 16s - loss: 6.5439 - 16s/epoch - 4ms/step
Epoch 8/10
4575/4575 - 16s - loss: 6.4454 - 16s/epoch - 4ms/step
Epoch 9/10
4575/4575 - 17s - loss: 6.3665 - 17s/epoch - 4ms/step
Epoch 10/10
4575/4575 - 18s - loss: 6.3003 - 18s/epoch - 4ms/step


In [65]:
modelMSE.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_23 (Conv1D)          (None, 8, 32)             96        
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, 2, 32)             0         
 g1D)                                                            
                                                                 
 global_average_pooling1d_5  (None, 32)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 flatten_6 (Flatten)         (None, 32)                0         
                                                                 
 dense_12 (Dense)            (None, 128)               4224      
                                                                 
 dense_13 (Dense)            (None, 1)                

In [75]:
test_predict_MSE = modelMSE.predict(val_x)

9149/9149 [==============================] - 17s 2ms/step


In [76]:
i =(len(x_command)-time_steps) - val_x.shape[0]%(len(x_command)-time_steps) -1
mean_3std = 0
for j in range(len(test_predict_MSE)):
    if i%len(x_command) == 0:
        i = time_steps
    if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
            mean_3std += 1
    i += 1
mean_3std/len(test_predict_MSE)

0.9881951489440192

radjeno u vise iteracija sa razlicitim vrednostima zbog vremenske kompleksnosti

In [80]:
filter_size = [16,32,64]
kernel_size_1 = [3,4]
kernel_size_2 = [3,4]
pooling_size = [3,4]
batch_size = [128,256]
#pool_type = [MaxPooling1D(pool_size = 3),MaxPooling1D(pool_size = 4), AveragePooling1D(pool_size = 3),AveragePooling1D(pool_size = 4)]
#global_pool_type = [GlobalAveragePooling1D(), GlobalMaxPool1D()]
time_steps_list = [8,10]

In [86]:
best_skor = float('-inf')
best_filter = None
best_kernel_1 = None
best_kernel_2 = None
best_pool = None
best_batch = None
#best_pool_type = None
#best_global_pool_type = None
best_step = None
for step in time_steps_list:
    
    #delimo podatke na osnovu velicine buffera
    input_shape = (step,1)

    for i in range(len(train_files_names)):
        exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, step)')

    train_X = np.asarray(train_X_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

    train_y = np.asarray(train_y_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')  
    
    #uzimamo percent_val sa kraja train_X skupa 
    val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
    val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

    train_X = train_X[:len(train_X)-val_x.shape[0]]
    train_y = train_y[:len(train_y)-val_x.shape[0]]
    
    for batch in batch_size:        
        for filt in filter_size:
            for kernel_1 in kernel_size_1:
                for kernel_2 in kernel_size_2:
                    for pool in pooling_size:
                        keras.utils.set_random_seed(7)
                        optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

                        modelMSE = Sequential()
                        modelMSE.add(Input(shape=input_shape))
                        modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_1,padding='same', activation=LeakyReLU()))
                        #modelMSE.add(Conv1D(filters=32, kernel_size=2, activation=LeakyReLU()))
                        modelMSE.add(MaxPooling1D(pool_size=pool))
                        modelMSE.add(Conv1D(filters=filt, kernel_size=kernel_2,padding='same', activation=LeakyReLU()))
                        #modelMSE.add(Conv1D(filters=64, kernel_size=2, activation=LeakyReLU()))
                        modelMSE.add(GlobalAveragePooling1D())
                        modelMSE.add(Flatten())
                        modelMSE.add(Dense(128, activation=LeakyReLU()))
                        modelMSE.add(Dense(units = 1, activation=LeakyReLU()))
                        modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                        modelMSE.fit(train_X, train_y, epochs = 10, batch_size = batch, verbose = 2, shuffle = False)
                        test_predict_MSE = modelMSE.predict(val_x)



                        #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                        #radimo evaluaciju na mean 3 sigma testu
                        i =(len(x_command)-time_steps) - val_x.shape[0]%(len(x_command)-time_steps) -1
                        mean_3std = 0
                        for j in range(len(test_predict_MSE)):
                            if i%len(x_command) == 0:
                                i = step
                            if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                    mean_3std += 1
                            i += 1



                        if mean_3std > best_skor:
                            best_skor = mean_3std
                            best_filter = filt
                            best_kernel_1 = kernel_1
                            best_kernel_2 = kernel_2
                            best_pool = pool
                            best_batch = batch
                            #best_pool_type = ptype
                            #best_global_pool_type = gptype
                            best_step = step

                            print('best_skor: ', best_skor)
                            print('best_filter: ', best_filter)
                            print('best_kernel_1: ', best_kernel_1)
                            print('best_kernel_2: ', best_kernel_2)
                            print('best_batch', best_batch)
                            print('best_step', best_step)
                            print('best_pool', best_pool)
                            #print('best_pool_type', best_pool_type)
                            #print('best_global_pool_type',best_global_pool_type)
                                    
print('best_skor: ', best_skor)
print('best_filter: ', best_filter)
print('best_kernel_1: ', best_kernel_1)
print('best_kernel_2: ', best_kernel_2)
print('best_batch', best_batch)
print('best_step', best_step)
print('best_pool', best_pool)
#print('best_pool_type', best_pool_type)
#print('best_global_pool_type',best_global_pool_type)

Epoch 1/10
9149/9149 - 24s - loss: 12.7065 - 24s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 23s - loss: 8.5592 - 23s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 23s - loss: 7.6903 - 23s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 23s - loss: 7.2565 - 23s/epoch - 3ms/step
Epoch 5/10
9149/9149 - 23s - loss: 6.9664 - 23s/epoch - 3ms/step
Epoch 6/10
9149/9149 - 23s - loss: 6.7475 - 23s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 23s - loss: 6.5673 - 23s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 23s - loss: 6.4109 - 23s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 23s - loss: 6.2799 - 23s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 23s - loss: 6.1725 - 23s/epoch - 3ms/step
9149/9149 [==============================] - 15s 2ms/step
best_skor:  289013
best_filter:  16
best_kernel_1:  3
best_kernel_2:  3
best_batch 128
best_step 8
best_pool 3
Epoch 1/10
9149/9149 - 25s - loss: 9.2969 - 25s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 23s - loss: 4.2920 - 23s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 24s - loss: 3.2638 - 24s/e

Epoch 4/10
9149/9149 - 26s - loss: 2.7532 - 26s/epoch - 3ms/step
Epoch 5/10
9149/9149 - 26s - loss: 2.6427 - 26s/epoch - 3ms/step
Epoch 6/10
9149/9149 - 26s - loss: 2.5620 - 26s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 27s - loss: 2.4997 - 27s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 26s - loss: 2.4548 - 26s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 26s - loss: 2.4142 - 26s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 27s - loss: 2.3719 - 27s/epoch - 3ms/step
9149/9149 [==============================] - 16s 2ms/step
Epoch 1/10
9149/9149 - 28s - loss: 7.5573 - 28s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 27s - loss: 3.3873 - 27s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 26s - loss: 2.7432 - 26s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 26s - loss: 2.5047 - 26s/epoch - 3ms/step
Epoch 5/10
9149/9149 - 26s - loss: 2.3887 - 26s/epoch - 3ms/step
Epoch 6/10
9149/9149 - 26s - loss: 2.3172 - 26s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 26s - loss: 2.2653 - 26s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 26s - lo

Epoch 7/10
9149/9149 - 31s - loss: 2.2216 - 31s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 31s - loss: 2.1838 - 31s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 31s - loss: 2.1498 - 31s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 30s - loss: 2.1168 - 30s/epoch - 3ms/step
9149/9149 [==============================] - 16s 2ms/step
Epoch 1/10
9149/9149 - 31s - loss: 6.1984 - 31s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 30s - loss: 3.0266 - 30s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 30s - loss: 2.7411 - 30s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 32s - loss: 2.5817 - 32s/epoch - 4ms/step
Epoch 5/10
9149/9149 - 34s - loss: 2.4619 - 34s/epoch - 4ms/step
Epoch 6/10
9149/9149 - 31s - loss: 2.3585 - 31s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 30s - loss: 2.3105 - 30s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 30s - loss: 2.2633 - 30s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 30s - loss: 2.2217 - 30s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 30s - loss: 2.1887 - 30s/epoch - 3ms/step
9149/9149 [===================

9149/9149 [==============================] - 16s 2ms/step
Epoch 1/10
4575/4575 - 16s - loss: 12.1848 - 16s/epoch - 3ms/step
Epoch 2/10
4575/4575 - 14s - loss: 8.3768 - 14s/epoch - 3ms/step
Epoch 3/10
4575/4575 - 15s - loss: 7.6395 - 15s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 15s - loss: 7.1975 - 15s/epoch - 3ms/step
Epoch 5/10
4575/4575 - 15s - loss: 6.9032 - 15s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 15s - loss: 6.6725 - 15s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 15s - loss: 6.4791 - 15s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 14s - loss: 6.3224 - 14s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 14s - loss: 6.1878 - 14s/epoch - 3ms/step
Epoch 10/10
4575/4575 - 14s - loss: 6.0767 - 14s/epoch - 3ms/step
9149/9149 [==============================] - 16s 2ms/step
Epoch 1/10
4575/4575 - 16s - loss: 8.9558 - 16s/epoch - 3ms/step
Epoch 2/10
4575/4575 - 15s - loss: 3.9725 - 15s/epoch - 3ms/step
Epoch 3/10
4575/4575 - 15s - loss: 3.0915 - 15s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 15s - loss: 2.

4575/4575 - 19s - loss: 2.4020 - 19s/epoch - 4ms/step
Epoch 7/10
4575/4575 - 19s - loss: 2.3351 - 19s/epoch - 4ms/step
Epoch 8/10
4575/4575 - 19s - loss: 2.2944 - 19s/epoch - 4ms/step
Epoch 9/10
4575/4575 - 19s - loss: 2.2580 - 19s/epoch - 4ms/step
Epoch 10/10
4575/4575 - 19s - loss: 2.2288 - 19s/epoch - 4ms/step
9149/9149 [==============================] - 16s 2ms/step
Epoch 1/10
4575/4575 - 20s - loss: 7.1445 - 20s/epoch - 4ms/step
Epoch 2/10
4575/4575 - 19s - loss: 3.1553 - 19s/epoch - 4ms/step
Epoch 3/10
4575/4575 - 19s - loss: 2.6779 - 19s/epoch - 4ms/step
Epoch 4/10
4575/4575 - 19s - loss: 2.4673 - 19s/epoch - 4ms/step
Epoch 5/10
4575/4575 - 19s - loss: 2.3612 - 19s/epoch - 4ms/step
Epoch 6/10
4575/4575 - 20s - loss: 2.3026 - 20s/epoch - 4ms/step
Epoch 7/10
4575/4575 - 20s - loss: 2.2530 - 20s/epoch - 4ms/step
Epoch 8/10
4575/4575 - 19s - loss: 2.2135 - 19s/epoch - 4ms/step
Epoch 9/10
4575/4575 - 20s - loss: 2.1855 - 20s/epoch - 4ms/step
Epoch 10/10
4575/4575 - 20s - loss: 2.1590

9149/9149 - 26s - loss: 2.5389 - 26s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 26s - loss: 2.4781 - 26s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 26s - loss: 2.4274 - 26s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 26s - loss: 2.3885 - 26s/epoch - 3ms/step
9149/9149 [==============================] - 15s 2ms/step
Epoch 1/10
9149/9149 - 26s - loss: 11.1591 - 26s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 25s - loss: 8.1910 - 25s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 25s - loss: 7.4257 - 25s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 25s - loss: 6.9958 - 25s/epoch - 3ms/step
Epoch 5/10
9149/9149 - 25s - loss: 6.7152 - 25s/epoch - 3ms/step
Epoch 6/10
9149/9149 - 25s - loss: 6.5103 - 25s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 25s - loss: 6.3508 - 25s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 26s - loss: 6.2210 - 26s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 27s - loss: 6.1158 - 27s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 25s - loss: 6.0228 - 25s/epoch - 3ms/step
9149/9149 [=============================

Epoch 10/10
9149/9149 - 30s - loss: 5.8292 - 30s/epoch - 3ms/step
9149/9149 [==============================] - 17s 2ms/step
Epoch 1/10
9149/9149 - 33s - loss: 6.5610 - 33s/epoch - 4ms/step
Epoch 2/10
9149/9149 - 32s - loss: 3.5120 - 32s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 32s - loss: 3.0017 - 32s/epoch - 3ms/step
Epoch 4/10
9149/9149 - 32s - loss: 2.7104 - 32s/epoch - 3ms/step
Epoch 5/10
9149/9149 - 31s - loss: 2.5205 - 31s/epoch - 3ms/step
Epoch 6/10
9149/9149 - 32s - loss: 2.4153 - 32s/epoch - 3ms/step
Epoch 7/10
9149/9149 - 32s - loss: 2.3298 - 32s/epoch - 3ms/step
Epoch 8/10
9149/9149 - 32s - loss: 2.2613 - 32s/epoch - 3ms/step
Epoch 9/10
9149/9149 - 32s - loss: 2.2046 - 32s/epoch - 3ms/step
Epoch 10/10
9149/9149 - 32s - loss: 2.1629 - 32s/epoch - 3ms/step
9149/9149 [==============================] - 17s 2ms/step
Epoch 1/10
9149/9149 - 31s - loss: 6.7469 - 31s/epoch - 3ms/step
Epoch 2/10
9149/9149 - 30s - loss: 3.6281 - 30s/epoch - 3ms/step
Epoch 3/10
9149/9149 - 30s - loss: 3.

4575/4575 - 13s - loss: 3.4935 - 13s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 13s - loss: 3.2447 - 13s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 13s - loss: 3.0579 - 13s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 13s - loss: 2.9160 - 13s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 13s - loss: 2.8131 - 13s/epoch - 3ms/step
Epoch 10/10
4575/4575 - 13s - loss: 2.7319 - 13s/epoch - 3ms/step
9149/9149 [==============================] - 15s 2ms/step
Epoch 1/10
4575/4575 - 16s - loss: 9.7647 - 16s/epoch - 4ms/step
Epoch 2/10
4575/4575 - 15s - loss: 4.3418 - 15s/epoch - 3ms/step
Epoch 3/10
4575/4575 - 15s - loss: 3.6183 - 15s/epoch - 3ms/step
Epoch 4/10
4575/4575 - 16s - loss: 3.2223 - 16s/epoch - 3ms/step
Epoch 5/10
4575/4575 - 15s - loss: 2.9435 - 15s/epoch - 3ms/step
Epoch 6/10
4575/4575 - 15s - loss: 2.7635 - 15s/epoch - 3ms/step
Epoch 7/10
4575/4575 - 15s - loss: 2.6481 - 15s/epoch - 3ms/step
Epoch 8/10
4575/4575 - 15s - loss: 2.5688 - 15s/epoch - 3ms/step
Epoch 9/10
4575/4575 - 15s - loss: 2.5091 

9149/9149 [==============================] - 17s 2ms/step
Epoch 1/10
4575/4575 - 21s - loss: 11.1452 - 21s/epoch - 5ms/step
Epoch 2/10
4575/4575 - 20s - loss: 8.1919 - 20s/epoch - 4ms/step
Epoch 3/10
4575/4575 - 20s - loss: 7.4338 - 20s/epoch - 4ms/step
Epoch 4/10
4575/4575 - 20s - loss: 6.9778 - 20s/epoch - 4ms/step
Epoch 5/10
4575/4575 - 20s - loss: 6.6785 - 20s/epoch - 4ms/step
Epoch 6/10
4575/4575 - 20s - loss: 6.4559 - 20s/epoch - 4ms/step
Epoch 7/10
4575/4575 - 20s - loss: 6.2976 - 20s/epoch - 4ms/step
Epoch 8/10
4575/4575 - 20s - loss: 6.1618 - 20s/epoch - 4ms/step
Epoch 9/10
4575/4575 - 20s - loss: 6.0596 - 20s/epoch - 4ms/step
Epoch 10/10
4575/4575 - 20s - loss: 5.9689 - 20s/epoch - 4ms/step
9149/9149 [==============================] - 17s 2ms/step
Epoch 1/10
4575/4575 - 22s - loss: 7.8973 - 22s/epoch - 5ms/step
Epoch 2/10
4575/4575 - 21s - loss: 3.7005 - 21s/epoch - 5ms/step
Epoch 3/10
4575/4575 - 21s - loss: 3.1428 - 21s/epoch - 5ms/step
Epoch 4/10
4575/4575 - 21s - loss: 2.

Zbog greske pri samom kraju testiranja arhitekture (falilo je ~8 modela), nastavljeno je u sledecoj petlji

In [154]:
gru_unit = [8,16]
unit1 = [8,16]
unit2 = [16]
batch_size = [128,256]
time_steps_list = [10]

In [155]:
best_skor = float('-inf')
best_gru = None
best_unit1 = None
best_unit2 = None
best_batch = None
best_step = None
for step in time_steps_list:
    
    input_shape = (step,1)

    for i in range(len(train_files_names)):
        exec(f'train_X_{i}, train_y_{i} = create_timeseries_split(razlika_trening_{i}, step)')

    train_X = np.asarray(train_X_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_X = np.append(train_X, train_X_{i}, axis=0)')

    train_y = np.asarray(train_y_0)
    for i in range(1,len(train_files_names)):
        exec(f'train_y = np.append(train_y, train_y_{i}, axis=0)')  
    
    val_x = train_X[-int(len(train_X)*percent_val)-1:-1]
    val_y = train_y[-int(len(train_X)*percent_val)-1:-1]

    train_X = train_X[:len(train_X)-val_x.shape[0]]
    train_y = train_y[:len(train_y)-val_x.shape[0]]
    
    for batch in batch_size:        
        for gru in gru_unit:
            for uni1 in unit1:
                for uni2 in unit2:

                    keras.utils.set_random_seed(7)
                    optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

                    model = Sequential()
                    model.add(GRU(units = gru, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
                    model.add(Dense(units = uni1, activation=keras.layers.LeakyReLU()))
                    model.add(Dense(units = uni2, activation=keras.layers.LeakyReLU()))
                    model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
                    model.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                    model.fit(train_X, train_y, epochs = 10, batch_size = batch, verbose = 2, shuffle = False)
                    test_predict_MSE = model.predict(val_x)

                    
                    
                    #zato sto smo uzeli 20% od kraja, ne pocinje indeks od samog pocetka rada masine
                    i =(61000-step) - val_x.shape[0]%(61000-step) -1
                    mean_3std = 0
                    for j in range(len(test_predict_MSE)):
                        if i%61000 == 0:
                            i = step
                        if sr_vrednost[i] - 3*std_dev[i] <= test_predict_MSE[j] <= sr_vrednost[i] + 3*std_dev[i]:
                                mean_3std += 1
                        i += 1


                    if mean_3std > best_skor:
                        best_skor = mean_3std
                        best_gru = gru
                        best_unit1 = uni1
                        best_unit2 = uni2
                        best_step = step
                        best_batch = batch

                        print('best_skor: ', best_skor,'best_gru: ', best_gru,'best_unit1: ', best_unit1,'best_unit2: ', best_unit2, 'best_batch', best_batch, 'best_step', best_step)
print('best_skor: ', best_skor)                    
print('best_gru: ', best_gru)
print('best_unit1: ', best_unit1)
print('best_unit2: ', best_unit2)
print('best_batch: ', best_batch)
print('best_step: ', best_step)

Epoch 1/10
9149/9149 - 44s - loss: 13.9486 - 44s/epoch - 5ms/step
Epoch 2/10
9149/9149 - 43s - loss: 3.8087 - 43s/epoch - 5ms/step
Epoch 3/10
9149/9149 - 44s - loss: 2.8849 - 44s/epoch - 5ms/step
Epoch 4/10
9149/9149 - 41s - loss: 2.6562 - 41s/epoch - 5ms/step
Epoch 5/10
9149/9149 - 39s - loss: 2.5781 - 39s/epoch - 4ms/step
Epoch 6/10
9149/9149 - 43s - loss: 2.5227 - 43s/epoch - 5ms/step
Epoch 7/10
9149/9149 - 43s - loss: 2.4709 - 43s/epoch - 5ms/step
Epoch 8/10
9149/9149 - 40s - loss: 2.4291 - 40s/epoch - 4ms/step
Epoch 9/10
9149/9149 - 41s - loss: 2.3962 - 41s/epoch - 4ms/step
Epoch 10/10
9149/9149 - 43s - loss: 2.3700 - 43s/epoch - 5ms/step
9149/9149 [==============================] - 17s 2ms/step
best_skor:  286255 best_gru:  8 best_unit1:  8 best_unit2:  16 best_batch 128 best_step 10
Epoch 1/10
9149/9149 - 47s - loss: 12.1176 - 47s/epoch - 5ms/step
Epoch 2/10
9149/9149 - 42s - loss: 3.1311 - 42s/epoch - 5ms/step
Epoch 3/10
9149/9149 - 43s - loss: 2.6658 - 43s/epoch - 5ms/step
Epo